In [3]:
import pynwb
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

In [4]:
file_name = '/Users/bendichter/Desktop/Buzsaki/SenzaiBuzsaki2017/YutaMouse41/YutaMouse41-150903.nwb'

In [5]:
nwb_io = NWBHDF5IO(file_name, mode='r')
nwb = nwb_io.read()

In [6]:
nwb2widget(nwb)

## Add custom tab

In [7]:
from nwbwidgets.view import neurodata_vis_spec
from nwbwidgets.misc import show_annotations

neurodata_vis_spec[pynwb.misc.AnnotationSeries]['star'] = lambda x: show_annotations(x, marker='*')

In [8]:
nwb2widget(nwb)